# TD Ameritrade API  
## Equity Quote

Install modules

In [125]:
import sys
# !{sys.executable} -m pip install

Load modules

In [1]:
from auth import TDClient
from us_holidays_cal import USHolidaysCalendar

In [2]:
import os
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from cassandra.cluster import Cluster

Set project directory and paths to credentials and tokens files

In [3]:
base_path = os.path.join(os.path.expanduser('~'), 'bakery')
creds_path = os.path.join(base_path, 'td_ameritrade/creds.json')
token_path = os.path.join(base_path, 'td_ameritrade/tokens.json')

Connect to Cassandra cluster

In [4]:
session = Cluster().connect()

Check if today is a weekend or holiday

In [19]:
current_year = date.today().year
cal = USHolidaysCalendar()
holidays = cal.holidays(
    start=date(current_year, 1, 1), 
    end=date(current_year, 12, 31)
)
holidays = [holiday.date() for holiday in holidays]

In [18]:
today = date.today()

is_weekend = today.weekday() in [5, 6]
if is_weekend:
    print('Today is not a weekday')
is_holiday = today in holidays
if is_holiday:
    print('Today is a holiday')

Today is a holiday


Get list of symbols from Cassandra table: bakery.symbols

In [129]:
def list_factory(colnames, rows):
    return [row[0] for row in rows]
session.row_factory = list_factory

query = "select symbol from bakery.symbols;"
symbols = session.execute(query, timeout=None)
symbols = [symbol for symbol in symbols]

Get authentication object

In [130]:
td_client = TDClient()
td_client.refresh_tokens()
td_auth = td_client.get_auth()

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


##### Get quotes for list of symbols

In [131]:
def chunker(ls, size):
    return (ls[pos:pos + size] for pos in range(0, len(ls), size))

In [133]:
for chunk in chunker(symbols, 100):
    
    symbols_str = ','.join(chunk)
    
    params = {
            'symbol': symbols_str
        }

    request_ts = int(datetime.now().timestamp()) * 1000
    
    r = requests.get(
            'https://api.tdameritrade.com/v1/marketdata/quotes',
            auth=td_auth,
            params=params
        )
    
    for s in r.json():
        values = (
            s, 
            request_ts,
            r.json()[s]['assetType'],
            r.json()[s]['assetMainType'],
            r.json()[s]['cusip'],
            r.json()[s]['assetSubType'],
            r.json()[s]['description'].replace("'", ""),
            r.json()[s]['bidPrice'],
            r.json()[s]['bidSize'],
            r.json()[s]['bidId'],
            r.json()[s]['askPrice'],
            r.json()[s]['askSize'],
            r.json()[s]['askId'],
            r.json()[s]['lastPrice'],
            r.json()[s]['lastSize'],
            r.json()[s]['lastId'],
            r.json()[s]['openPrice'],
            r.json()[s]['highPrice'],
            r.json()[s]['lowPrice'],
            r.json()[s]['bidTick'],
            r.json()[s]['closePrice'],
            r.json()[s]['netChange'],
            r.json()[s]['totalVolume'],
            r.json()[s]['quoteTimeInLong'],
            r.json()[s]['tradeTimeInLong'],
            r.json()[s]['mark'],
            r.json()[s]['exchange'],
            r.json()[s]['exchangeName'],
            r.json()[s]['marginable'],
            r.json()[s]['shortable'],
            r.json()[s]['volatility'],
            r.json()[s]['digits'],
            r.json()[s]['52WkHigh'],
            r.json()[s]['52WkLow'],
            r.json()[s]['nAV'],
            r.json()[s]['peRatio'],
            r.json()[s]['divAmount'],
            r.json()[s]['divYield'],
            r.json()[s]['divDate'],
            r.json()[s]['securityStatus'],
            r.json()[s]['regularMarketLastPrice'],
            r.json()[s]['regularMarketLastSize'],
            r.json()[s]['regularMarketNetChange'],
            r.json()[s]['regularMarketTradeTimeInLong'],
            r.json()[s]['netPercentChangeInDouble'],
            r.json()[s]['markChangeInDouble'],
            r.json()[s]['markPercentChangeInDouble'],
            r.json()[s]['regularMarketPercentChangeInDouble'],
            r.json()[s]['delayed'],
            r.json()[s]['realtimeEntitled'],        
        )

        session.execute(
            f'''
            INSERT INTO bakery.quotes_daily 
                (symbol, request_ts, asset_type, asset_main_type, cusip, asset_subtype, description, 
                bid_price, bid_size, bid_id, ask_price, ask_size, ask_id, last_price, last_size, 
                last_id, open_price, high_price, low_price, bid_tick, close_price, net_change, 
                total_volume, quote_time_in_long, trade_time_in_long, mark, exchange, exchange_name, 
                marginable, shortable, volatility, digits, high_52_wk, low_52_wk, nav, pe_ratio, 
                div_amount, div_yield, div_date, security_status, regular_market_last_price, 
                regular_market_last_size, regular_market_net_change, regular_market_trade_time_in_long, 
                net_percent_change_in_double, mark_change_in_double, mark_percent_change_in_double, 
                regular_market_percent_change_in_double, delayed, real_time_entitled)
            VALUES {values}
            '''
        )

In [108]:
r.json()

{'FCPT': {'assetType': 'EQUITY',
  'assetMainType': 'EQUITY',
  'cusip': '35086T109',
  'assetSubType': '',
  'symbol': 'FCPT',
  'description': 'Four Corners Property Trust, Inc. Common Stock',
  'bidPrice': 28.46,
  'bidSize': 400,
  'bidId': 'T',
  'askPrice': 28.48,
  'askSize': 100,
  'askId': 'K',
  'lastPrice': 28.46,
  'lastSize': 100,
  'lastId': 'U',
  'openPrice': 28.44,
  'highPrice': 28.48,
  'lowPrice': 28.1,
  'bidTick': ' ',
  'closePrice': 28.45,
  'netChange': 0.01,
  'totalVolume': 119096,
  'quoteTimeInLong': 1659728587913,
  'tradeTimeInLong': 1659728551008,
  'mark': 28.46,
  'exchange': 'n',
  'exchangeName': 'NYSE',
  'marginable': True,
  'shortable': True,
  'volatility': 0.0478,
  'digits': 2,
  '52WkHigh': 30.13,
  '52WkLow': 24.24,
  'nAV': 0.0,
  'peRatio': 23.3896,
  'divAmount': 1.33,
  'divYield': 4.67,
  'divDate': '2022-06-29 00:00:00.000',
  'securityStatus': 'Normal',
  'regularMarketLastPrice': 28.46,
  'regularMarketLastSize': 1,
  'regularMarketN